## This notebook is used to Debug functionalities

In [1]:
import torch
import torch.nn as nn
import tiktoken
from model import *
from dataset import *
from torch.utils.data import DataLoader

In [2]:
# Get gpt-2 tokenizer
tokenizer = tiktoken.get_encoding("gpt2")
vocab_size = tokenizer.n_vocab
print("Vocab size:", vocab_size)

Vocab size: 50257


## Read data from a test file

In [3]:
with open("../data/the-verdict.txt", "r") as file:
    raw_text = file.read()

print(len(raw_text), "characters")

20479 characters


## Create Dataset object

In [4]:
dataset = GPTDataset(raw_text, tokenizer, 126, 1) # input phrases of 126 tokens

inputs, labels = dataset[0]
print("Inputs shape:", inputs.shape)
print("Labels shape:", labels.shape)

Inputs shape: torch.Size([126])
Labels shape: torch.Size([126])


## Create DataLoader

In [5]:
dataloader = DataLoader(
    dataset = dataset,
    batch_size = 32,
    shuffle = True
)

dataiter = iter(dataloader)
inputs_batch, labels_batch = next(dataiter)

print("Input batch shape:", inputs_batch.shape)
print("Labels batch shape:", labels_batch.shape)

Input batch shape: torch.Size([32, 126])
Labels batch shape: torch.Size([32, 126])


In [7]:
args = ModelArgs(
    emb_dim = 768,
    num_heads = 2,
    context_length=126
)

In [9]:
# apply embedding
emb_dim = 768
embedding = nn.Embedding(vocab_size, emb_dim)
vectors = embedding(inputs_batch)

attention = MultiHeadAttention(args)
z = attention(vectors)
print("Context vector shape:", z.shape)

torch.Size([32, 2, 126, 126])
Context vector shape: torch.Size([32, 126, 768])
